# Stock capital gain calculation
## First in First out law (FIFO)

### - Import libraries

In [1]:
import pandas as pd
import numpy as np

### Extract required data 

In [2]:
# Read excel file
df = pd.read_excel("tax_2022.xlsx", sheet_name="Sheet2", skiprows=1)

# Set a filter for neccessary data
col_list = ['Code', 'Date', 'Type', 'Quantity', 'Total Value ($)']

# Filter the dataframe to keep the data required
df = df[col_list]

# Drop NaN rows
df.dropna(inplace=True)

# Drop duplicate name rows
filter = df['Type'] != 'Type'
df = df[filter]

# Change data type from string to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True) # Date : String to datetime format

# Sort data for Code names and event Date
df.sort_values(['Code', 'Date'], ascending=True, inplace=True) # Sorting multiple names [Code, Date]

# Reset the index numbers
df.reset_index(drop=True, inplace=True) # Reset index from 0
df['Unit Value ($)'] = df['Total Value ($)'] / df['Quantity']
df

,Code,Date,Type,Quantity,Total Value ($),Unit Value ($)
0,ETHI,2020-06-11,Buy,51,549.23,10.769216
1,ETHI,2021-06-04,Buy,83,994.68,11.984096
2,ETHI,2021-06-09,Buy,83,998,12.024096
3,ETHI,2021-06-18,Buy,80,993.2,12.415
4,ETHI,2021-06-23,Buy,80,997.16,12.4645
5,ETHI,2021-07-20,Buy,82,998.3,12.17439
6,ETHI,2021-09-21,Sell,-459,-5751.22,12.529891
7,ETHI,2021-10-04,Buy,803,9993.2,12.444832
8,ETHI,2021-10-28,Buy,239,2998.26,12.545021
9,ETHI,2022-04-14,Sell,-86,-993.88,11.556744


### Capital Gain Calculator: FIFO

In [3]:
import logging
from collections import deque
import math

class Trans:
    datetime=None
    amount=None
    price=None
 
    def __init__(self, datetime, amount, price):
        self.datetime=datetime
        self.amount=amount
        self.price=price
     
    def getInfo(self):
        return(str(self.datetime)+"; "+
                str(self.amount)+"; "+
                str(self.price))+"; "
 
def balanceFifo(all_trans):
 
    qTransactions = deque() 
    trans_result = list()
 
    for t in all_trans:
        #Add first element to the queue
        if len(qTransactions)==0:
            #logging.debug('Added the first element: %s',t.getInfo())
            qTransactions.append(t)
            continue
 
        while (t.amount!=0 and len(qTransactions)>0):
            #investigate the first element from the queue
            tq=qTransactions.popleft()
            #the same type of transaction: both sell or both buy
            if tq.amount*t.amount>0:
                #return the first element back to the same place
                qTransactions.appendleft(tq)
                #add the new element to the list
                qTransactions.append(t)
                #logging.debug('Added: %s',t.getInfo())
                break
             
            #contrary transactions: (sell and buy) or (buy and sell) 
            if tq.amount*t.amount<0:
                #logging.debug('Transaction : %s',t.getInfo())
                #logging.debug('... try to balance with: %s',tq.getInfo())
 
                #The element in the queue have more units and takes in the current transaction
                if abs(tq.amount)>abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                            math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount of the element in the queue
                    tq.amount=tq.amount+t.amount
                    #return the element back to the same place
                    qTransactions.appendleft(tq)
                    #logging.debug('Removed transaction: %s',t.getInfo())
                    #the transaction has been balanced, take a new transaction
                    break
                 
                #The element from the queue and transaction have the same amount of units
                if abs(tq.amount)==abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                                math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount in the transaction 
                    t.amount=0
                    #logging.debug('Balanced, removed transaction: %s',t.getInfo())
                    #logging.debug('Balanced, removed from the queue: %s',tq.getInfo())
                    #the transaction has been balanced, take a new transaction
                    continue
                    
                #The transaction has more units
                if abs(tq.amount)<abs(t.amount):
                    #update the units in transaction, (remove element from the queue)
                    t.amount=t.amount+tq.amount
                    result = insertTransaction(tq.datetime,t.datetime,tq.amount,tq.price,t.price)
                    trans_result.append(result)
                    #logging.debug('Removed from queue: %s',tq.getInfo())
                     
                    #the transaction has not been balanced, 
                    #take a new element from the queue (t.amount>0)
                    continue
                 
        #We have unbalanced transaction but the queue is empty            
        if (t.amount!=0 and len(qTransactions)==0):
            #Add unbalanced transaction to the queue
            #The queue changes polarisation
            qTransactions.append(t)
            #logging.debug('Left element: %s',t.getInfo())
     
     
    #If something remained in the queue, treat it as open or part-open transactions
    while (len(qTransactions)>0):
        tq=qTransactions.popleft()
        #logging.debug('Remained on list transaction: %s',tq.getInfo())
        
    return trans_result
 
def insertTransaction(dateStart,dateEnd,amount,priceStart,priceEnd):
    #print("Bought={}, sold={},  amount={}, buy price={}, sell_price={}, gain={}".\
    #        format(dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)))
    result = [dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)]
    return result

### Calculate Capital Gain for Taxation using FIFO calculator

In [4]:
# Extract code names, numbers and counts
code = df['Code'].value_counts()
print("size: ", code.size, "index: ", code.index, "value: ", code.values)

size:  6 index:  Index(['NDQ', 'OSH', 'ETHI', 'IOZ', 'SYI', 'MQG'], dtype='object') value:  [22 12 11  4  3  2]


In [10]:
# For loop for each Code strings 

start_date = pd.to_datetime("2021-06-30")
end_date = pd.to_datetime("2022-07-01")
sum_df = []

for j in range(0, code.size):
    # Seperate dataframe by Code name
    mask_code = df['Code'] == code.index[j]
    df_mask = df[mask_code]
    df_mask.reset_index(drop=True, inplace=True)
    print("All Transactions for", code.index[j])
    print(df_mask)
    
    # Capital gain caluculation FIFO
    trans_list=list()
    for i in range(0, df_mask.shape[0]):
        trans = Trans(df_mask['Date'][i].date(), df_mask['Quantity'][i], df_mask['Unit Value ($)'][i])
        trans_list.append(trans)
    trans_result = balanceFifo(trans_list)
    df_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])

    # Filtering sold date for 2021-2022 Capital Gain
    mask_time = (df_capital['Date Sold'] > start_date) & (df_capital['Date Sold'] < end_date)
    df_filtered = df_capital[mask_time]

    print("Financial Year Capital gain for", code.index[j])
    print(df_filtered)
    
    # Summation of capita gains (Total: any loses, discounts not included, Net: including loses or discounts)
    sum_total = np.sum(x for x in df_filtered['Capital Gain'] if x > 0)
    sum_net = np.sum(x for x in df_filtered['Capital Gain'])
    
    #For net capital gain, discounts for 'over 1 year stocks' should be applied
    diff = df_filtered['Date Sold'] - df_filtered['Date Purchased'] # How many days holding stocks
    discount = 0 # Amount of discount
    
    # If you hold stocks more than a year, you get half of discount on capital gain for taxation
    for y in range(0, diff.size):
        if (int(diff.values[y]/8.64e+13) > 365): 
            discount = discount + df_filtered['Capital Gain'][y] / 2
            sum_net = sum_net - discount
        else: 
            sum_net = sum_net
        
    print("Total capital gain for", code.index[j], ": ", round(sum_total, 2), "AUD")
    print("Net capital gain for", code.index[j], ": ", round(sum_net, 2), "AUD")
    
    # Append data into Summary dataframe
    a = [code.index[j], sum_total, sum_net]
    sum_df.append(a)

All Transactions for NDQ
   Code       Date  Type Quantity Total Value ($) Unit Value ($)
0   NDQ 2020-10-28   Buy       55          1476.4      26.843636
1   NDQ 2020-11-13   Buy       47         1278.13      27.194255
2   NDQ 2020-12-29   Buy       20           561.6          28.08
3   NDQ 2021-01-14   Buy      179         4986.16      27.855642
4   NDQ 2021-02-23   Buy       19          530.58      27.925263
5   NDQ 2021-03-01   Buy        8          224.16          28.02
6   NDQ 2021-03-22   Buy       17          473.41      27.847647
7   NDQ 2021-05-07   Buy       34          989.02      29.088824
8   NDQ 2021-05-25   Buy       20             586           29.3
9   NDQ 2021-07-02   Buy       26          809.82      31.146923
10  NDQ 2021-07-29   Buy       30           975.5      32.516667
11  NDQ 2021-07-30   Buy       31          996.17      32.134516
12  NDQ 2021-08-03   Buy       61          1982.8      32.504918
13  NDQ 2021-09-14   Buy      154         5169.32      33.567013


C:\Users\reals\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\reals\AppData\Local\Temp\ipykernel_3000\403946184.py:31: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum_total = np.sum(x for x in df_filtered['Capital Gain'] if x > 0)
C:\Users\reals\AppData\Local\Temp\ipykernel_3000\403946184.py:32: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum_net = np.sum(x for x in df_filtered['Capital Gain'])
C:\Users\

In [11]:
# Print summary dataframe for total and net capital gain
summary = pd.DataFrame(sum_df, columns = ['Code', 'Total gain', 'Net gain'])
print(summary)
print("Total Capital Gain: ", summary['Total gain'].sum().round(), "AUD")
print("Net Capital Gain: ", summary['Net gain'].sum().round(), "AUD")

   Code   Total gain     Net gain
0   NDQ  2191.089757  2106.540000
1   OSH     0.000000  -105.328149
2  ETHI   220.650000    23.001694
3   IOZ    79.252400    74.030000
4   SYI     0.000000     0.000000
5   MQG     0.000000     0.000000
Total Capital Gain:  2491.0 AUD
Net Capital Gain:  2098.0 AUD


In [7]:
summary.to_excel('tax_return_2022_AU.xlsx', sheet_name = 'AU_CG')